In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/Pytorch_and_cv/Assingment/Artenal/TRAIN/workspace/training_demo/Yolo-v4-DeepSort

/content/drive/MyDrive/Pytorch_and_cv/Assingment/Artenal/TRAIN/workspace/training_demo/Yolo-v4-DeepSort


In [5]:
import os
# comment out below line to enable tensorflow logging outputs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import time
import tensorflow as tf

import core.utils as utils

from tensorflow.python.saved_model import tag_constants

import cv2
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
# deep sort imports
from deep_sort import preprocessing, nn_matching
from deep_sort.detection import Detection
from deep_sort.tracker import Tracker
from tools import generate_detections as gdet

In [6]:
def get_prediction(pred_bbox,threshold=.45):
    boxs,scors,classe = [],[],[]
    boxes = np.array(pred_bbox["detection_boxes"][0])
    classes = np.array(pred_bbox["detection_classes"][0])
    scores = np.array(pred_bbox["detection_scores"][0])
    
    for box,scor,cls in zip(boxes,scores,classes):
        if scor > threshold-.20:
            boxs.append(box),scors.append(scor),classe.append(cls)
#     if len(classe)<num_detection:
#         for appnd_extra in range(len(classe),num_detection):
#             boxs.append([0,0,0,0]),scors.append(0),classe.append(1)
    return tf.constant(boxs,dtype=np.float32),tf.constant(scors,dtype=np.float32),tf.constant(classe,dtype=np.float32),tf.constant(len(classe))

In [29]:
max_cosine_distance = 0.4
nn_budget = None
nms_max_overlap = 1.0

# initialize deep sort
model_filename = 'deep_sort_model/mars-small128.pb'
encoder = gdet.create_box_encoder(model_filename, batch_size=1)
# calculate cosine distance metric
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
# initialize tracker
tracker = Tracker(metric)

# load configuration for object detector
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
STRIDES, ANCHORS, NUM_CLASS, XYSCALE = utils.load_config()

saved_model_loaded = tf.saved_model.load('/content/drive/MyDrive/Pytorch_and_cv/Assingment/Artenal/TRAIN/workspace/training_demo/exported-models/ssd_mobilenet/saved_model/', tags=[tag_constants.SERVING])
infer = saved_model_loaded.signatures['serving_default']

/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [19]:
from google.colab.patches import cv2_imshow

In [21]:
colors = {'aut':[227,172,7], 'bus':[17,17,195], 'car':[0,103,82], 'lcv':[170,84,8], 'mot':[46,46,46], 'mul':[192,3,3], 'tra':[18,255,24], 'tru':[52,2,182]}

In [32]:
video_path = "/content/drive/MyDrive/Pytorch_and_cv/Assingment/Artenal/TRAIN/workspace/training_demo/Tests/INDTraffic.mp4" # webcam
# begin video capture
cap = cv2.VideoCapture(video_path)

out = None
save = True


save_video_path = "/content/drive/MyDrive/Pytorch_and_cv/Assingment/Artenal/TRAIN/workspace/training_demo/Tests/INDTraffic12.mp4"

if save:
    # by default VideoCapture returns float instead of int
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    out = cv2.VideoWriter(save_video_path,cv2.VideoWriter_fourcc(*'DIVX'), 29, (width, height))
count_cars = []

# while video is running
while True:
    ret, frame = cap.read()

    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    else:
        print('Video has ended!')
        break


    image_data = frame[np.newaxis, ...].astype(np.float32)
    
    start_time = time.time()

    batch_data = tf.constant(image_data,dtype=np.uint8)
    pred_bbox = infer(batch_data)


    boxes, scores, classes, valid_detections = get_prediction(pred_bbox,threshold=.48)

    # convert data to numpy arrays and slice out unused element
    bboxes = boxes.numpy()
    scores = scores.numpy()
    classes = classes.numpy()
    
    # format bounding boxes from normalized ymin, xmin, ymax, xmax ---> xmin, ymin, width, height
    original_h, original_w, _ = frame.shape
    bboxes = utils.format_boxes(bboxes, original_h, original_w)



    # read in all class names from config
    class_names = {1: 'auto', 2: 'bus', 3: 'car', 4: 'lcv', 5: 'motorcycle', 6: 'multiaxle', 7: 'tractor', 8: 'truck'}
    
    # loop through objects and use class index to get class name, allow only classes in allowed_classes list
    names = []
    for cls in classes:
        class_name = class_names[cls]
        names.append(class_name)
    names = np.array(names)

    # encode yolo detections and feed to tracker
    features = encoder(frame, bboxes)
    
    detections = [Detection(bbox, score, class_name, feature) for bbox, score, class_name, feature in zip(bboxes, scores, names, features)]


    # run non-maxima supression
    boxs = np.array([d.tlwh for d in detections])
    scores = np.array([d.confidence for d in detections])
    classes = np.array([d.class_name for d in detections])
    indices = preprocessing.non_max_suppression(boxs, classes, nms_max_overlap, scores)
    detections = [detections[i] for i in indices]       
    # Call the tracker
    tracker.predict()
    tracker.update(detections)

    # update tracks
    for track in tracker.tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue 
        bbox = track.to_tlbr()
        class_name = track.get_class()
        # draw bbox on screen
        color = colors[class_name[0:3]]
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), color, 2)
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1]-30)), (int(bbox[0])+(len(class_name)+len(str(track.track_id)))*17, int(bbox[1])), color, -1)
        cv2.putText(frame, class_name + "-" + str(track.track_id),(int(bbox[0]), int(bbox[1]-10)),0, 0.75, (255,255,255),2)
        
        count_cars.append(class_name+str(track.track_id))
    
    
    result = np.asarray(frame)
    result = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
    # put total vehicle
    total_count = len(set(count_cars))
    cv2.putText(result,"Total vehicle count "+str(total_count),(10,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,50,50),2)

#     cv2.imshow("Output Video", result)
    # if output flag is set, save video file
    if save:
        out.write(result)

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyAllWindows()

KeyboardInterrupt: ignored

In [ ]:
frame = cv2.imread("G:/My Drive/Pytorch_and_cv/Assingment/Artenal/TRAIN/workspace/training_demo/Tests/images.jpg")
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)



image_data = frame[np.newaxis, ...].astype(np.float32)

start_time = time.time()

batch_data = tf.constant(image_data,dtype=np.uint8)
pred_bbox = infer(batch_data)


boxes = pred_bbox["detection_boxes"]
pred_conf = pred_bbox["detection_scores"]
        

In [ ]:
boxes, scores, classes, valid_detections = get_prediction(pred_bbox,threshold=.45,num_detection=50)

In [ ]:
# convert data to numpy arrays and slice out unused elements
num_objects = valid_detections.numpy()
bboxes = boxes.numpy()
bboxes = bboxes[0:int(num_objects)]
scores = scores.numpy()
scores = scores[0:int(num_objects)]
classes = classes.numpy()
classes = classes[0:int(num_objects)]


In [ ]:
# format bounding boxes from normalized ymin, xmin, ymax, xmax ---> xmin, ymin, width, height
original_h, original_w, _ = frame.shape
bboxes = utils.format_boxes(bboxes, original_h, original_w)

# store all predictions in one parameter for simplicity when calling functions
pred_bbox = [bboxes, scores, classes, num_objects]

# read in all class names from config
class_names = class_name

# by default allow all classes in .names file
# allowed_classes = list(class_names.values())

# custom allowed classes (uncomment line below to customize tracker for only people)
allowed_classes = ["auto",'bus','car','lcv','motorcycle','multiaxle','tractor','truck']


In [ ]:
names = []
for cls in classes:
    class_name = class_names[int(cls)]
    names.append(class_name)
names = np.array(names)
count = len(names)

In [ ]:
cv2.destroyAllWindows()